# Notebook rilut

### Bikin jelek, hapus aja

- colorCode
- total_spent, 
- customerID
- articleID
- orderDate
- productGroup (karena di-impute paksa dengan 0? atau emang ga penting?)

### Jangan dihapus

- rrp, sizeCode

### Ga pengaruh / pengaruh dikit banget

- months (labelencoder), kali dihapus, dibinerin, atau diapain
- mmdd (labelencoder), kali dihapus, dibinerin, atau diapain

kalo moths/mmdd diilangin, berkurang 3. tapi kalo months & mmdd dihilangin sekaligus, berkurang 10.

### Missing values

rrp, productGroup

### Catatan:

Logistic Regression lama banget. itu pun solvernya udah diubah. kalau masih mau pake, iternya kurangin kali?

### TODO:

- pakai PolynomialFeatures (di selain fitur harga (?) (selain price, rrp, after_voucher, unit_price, usual_unit_price, price_diff, etc))
- ekstrak fitur karena fitur adalah segalanya

In [1]:
from __future__ import division

label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from operator import itemgetter
#from sklearn.externals import joblib

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

def feature_importances(Xdf, rf_model):
    pair=zip(Xdf.columns, rf_model.feature_importances_)
    t1=[i for i in pair]; t1
    return sorted(t1, key=itemgetter(1))

In [2]:
# Pakai make_datasets_train_only. Buat sendiri juga boleh

df=pd.read_csv('train_dikit.csv')

if 'deviceID' in df.columns:
    del df['deviceID']
    
del df["colorCode"]
del df['customerID']
del df['articleID']
del df['orderDate']
#del df['productGroup']

df.head()

,orderID,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,paymentMethod,returnQuantity,...,price_diff,after_voucher,order_order,choice_order,articleID_prob,colorCode_prob,sizeCode_prob,customerID_prob,ac_prob,as_prob
0,a1000001,44,3.0,1,10.0,29.984375,0,0.0,BPRG,0,...,0.000000,30.0,1,1,0.421143,0.521484,0.516113,0.0,0.421053,0.421053
1,a1000001,44,3.0,1,20.0,40.000000,0,0.0,BPRG,0,...,5.000000,30.0,2,2,0.506836,0.492432,0.516113,0.0,0.475410,0.475410
2,a1000002,38,8.0,1,35.0,50.000000,0,0.0,BPRG,0,...,10.000000,85.0,1,1,0.642090,0.455322,0.535645,0.5,0.616505,0.616505
3,a1000002,38,8.0,1,50.0,50.000000,0,0.0,BPRG,1,...,14.990002,85.0,2,2,0.694336,0.479492,0.535645,0.5,0.713450,0.713450
4,a1000003,42,8.0,1,10.0,36.000000,0,0.0,PAYPALVC,0,...,0.000000,60.0,1,1,0.600586,0.606934,0.526367,0.0,0.573034,0.573034


In [3]:
df.customerID_prob[df.customerID_prob>1]=1

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [4]:
# Metode

In [5]:
#Konversi kategori/object ke angka (ordinal)
print "Konversi kategori/object ke angka (ordinal):"

#Cari kolom yang tipenya object, bukan integer maupun float
object_columns=df.loc[:, df.dtypes == object].columns


for col in object_columns:
    print col
    le=LabelEncoder()
    # Konversi deh
    df[col]=le.fit_transform(df[col])
    
if 'orderID' in df.columns:
    del df['orderID']

Konversi kategori/object ke angka (ordinal):
orderID
sizeCode
voucherID


C:\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


paymentMethod


In [10]:
# Turunin presisi, iseng-iseng aja
float_64_columns = df.loc[:, df.dtypes == np.float64].columns
print "float_64_columns", float_64_columns.tolist()
for col in float_64_columns:
    df[col] = df[col].astype(np.float32)

df.ac_prob=df.ac_prob.astype(np.float16)
df.as_prob=df.as_prob.astype(np.float16)
df.articleID_prob=df.articleID_prob.astype(np.float16)
df.colorCode_prob=df.colorCode_prob.astype(np.float16)
df.sizeCode_prob=df.sizeCode_prob.astype(np.float16)
df.customerID_prob=df.customerID_prob.astype(np.float16)
#
    
int_64_columns = df.loc[:, df.dtypes == np.int64].columns
for col in int_64_columns:
    df[col] = df[col].astype(np.int32)
    

# Impute
df.rrp.fillna(df.rrp.median(), inplace=True)
df.productGroup.fillna(0, inplace=True)

print ''

Xdf=df.drop([label, 'productGroup'], axis=1)
print "Kolom yang dipakai: ", Xdf.columns.tolist()

X=Xdf.values
y=df[label].values

# Cara cek Nan
#Xdf.columns-Xdf.dropna(axis=1).columns

float_64_columns []

Kolom yang dipakai:  ['sizeCode', 'quantity', 'price', 'rrp', 'voucherID', 'voucherAmount', 'paymentMethod', 'order_total', 'customer_budget', 'expense_ratio', 'unit_price', 'usual_unit_price', 'price_diff', 'after_voucher', 'order_order', 'choice_order', 'articleID_prob', 'colorCode_prob', 'sizeCode_prob', 'customerID_prob', 'ac_prob', 'as_prob']


In [11]:
# Biasa, Non-Polynomial

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.218, random_state=42)
lr=LinearRegression(n_jobs=-1, normalize=True)
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
#y_pred=np.array([round(yp+0.00049) for yp in y_pred], dtype=np.int8)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[117406  49992      9      0      0      0]
 [ 31536 149268      7      0      0      0]
 [     9    537      1      0      0      0]
 [     0     18      9      0      0      0]
 [     0      1      1      0      0      0]
 [     0      0      6      0      0      0]]
82176


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
X=Xdf.values
y=df.returnQuantity.values
poly = PolynomialFeatures(2)
X2 = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.218, random_state=42)

In [ ]:
lr=LinearRegression(n_jobs=-1, normalize=True)
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
#y_pred=np.array([round(yp+0.00049) for yp in y_pred], dtype=np.int8)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[119152  48249      6      0      0      0]
 [ 31962 148764     82      3      0      0]
 [    16    349    182      0      0      0]
 [     0      7     15      5      0      0]
 [     0      1      0      1      0      0]
 [     0      1      1      3      1      0]]
80739


In [ ]:
0

In [ ]:
"""df.unit_price=df.unit_price.astype(np.float32,)
df.usual_unit_price=df.usual_unit_price.astype(np.float32)
df.price_diff=df.price_diff.astype(np.float32)
df.after_voucher=df.after_voucher.astype(np.float32)
df.rrp=df.rrp.astype(np.float32)
df.customer_budget=df.customer_budget.astype(np.float32)
df.order_total=df.order_total.astype(np.float32)
df.voucherAmount=df.voucherAmount.astype(np.float32)
df.price=df.price.astype(np.float32)
df.productGroup=df.productGroup.astype(np.int16)

df.info()"""